#Preprocessing

In [1]:
import os
from zipfile import ZipFile as zf

with zf('data.zip','r') as zin:
    zin.extractall('./data/')

In [61]:
os.chdir('./data/')

In [62]:
import pandas as pd
import py_midicsv as pm

paths = pd.read_csv('./maestro-v3.0.0.csv')

paths['midi_filename'] = paths['midi_filename'].apply(lambda s:s.split('/')[1])
songs = paths['midi_filename']
paths

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2018,MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_201...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
1,Alban Berg,Sonata Op. 1,train,2008,MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AU...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
2,Alban Berg,Sonata Op. 1,train,2017,MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AU...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AU...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508
...,...,...,...,...,...,...,...
1271,Wolfgang Amadeus Mozart,"Sonata in F Major, K280",test,2004,MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AU...,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,241.470442
1272,Wolfgang Amadeus Mozart,"Sonata in F Major, K280",train,2004,MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AU...,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,114.696243
1273,Wolfgang Amadeus Mozart,"Sonata in F Major, K533",validation,2004,MIDI-Unprocessed_SMF_12_01_2004_01-05_ORIG_MID...,2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORI...,1139.198478
1274,Wolfgang Amadeus Mozart,"Sonata in F Major, K533/K494",validation,2018,MIDI-Unprocessed_Recital17-19_MID--AUDIO_17_R1...,2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...,1068.751602


In [30]:
os.mkdir('../csv/')
for song in songs:
    # Load the MIDI file and parse it into CSV format
    csv_data = pm.midi_to_csv(f'./{song}')
    with open(f'../csv/{song}.csv', "w") as f:
        f.writelines(csv_data)

#UnderstandingData

In [63]:
os.chdir('../')

In [64]:
from sklearn.decomposition import PCA

folder_path = './csv'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

data = []

for file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
    df.columns = ['channel','time','action','glide','pitch','pedal']
    df = df.dropna()
    df.drop(['channel','glide','action'],axis=1,inplace=True)
    clf = PCA(n_components=0.9)
    new_ft = clf.fit_transform(df.transpose().to_numpy())
    df = pd.DataFrame(new_ft.T,columns=['pca1','pca2','pca3'])
    df['midi_filename'] = file
    # df = df.set_index(['song'])
    data.append(df)

df = pd.concat(data)

C:\Users\W\AppData\Local\Temp\ipykernel_1584\3940857365.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
Skipping line 4: expected 6 fields, saw 7

C:\Users\W\AppData\Local\Temp\ipykernel_1584\3940857365.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
Skipping line 4: expected 6 fields, saw 7

C:\Users\W\AppData\Local\Temp\ipykernel_1584\3940857365.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
Skipping line 4: expected 6 fields, saw 7

C:\Users\W\AppData\Local\Temp\ipykernel_1584\3

IndexError: list index out of range

In [122]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

X = MinMaxScaler().fit_transform(df)
y = np.random.choice(['Bach'],) # TODO: Append Y labels to each song in the beginning of the notebook (also include other composers...)

#WithoutML

#NoFreeLunch